<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 03 Building The Prediction Model

### Riccardo Spolaor (864877)

</center>

Modifico il dataframe in modo da poter ottenere un modello adatto alla predizione:
* Rinomino ***Winner*** in ***PlayerA*** e ***Loser*** in ***PlayerB***;
* Rinomino tutte le colonne con informazioni legate al vincitore e al perdente in modo da riflettere il legame ai nuovi nomi assegnati;
* Creo la colonna ***Winner*** che assume i valori:
    * 0 se ha vinto ***PlayerA***;
    * 1 se ha vinto ***PlayerB***;
* Assegno a ***Winner*** tutti valori 0;
* Duplico il dataframe e faccio lo swapping tra tutte le colonne con informazioni sul Giocatore A e quelle con informazioni sul Giocatore B;
* Assegno il valore 1 a ogni entry della colonna ***Winner*** del nuovo dataframe;
* Unisco i due dataframe.

In [779]:
import pandas as pd

In [780]:
dataframe = pd.read_csv('./datasets/dataframe05.csv', low_memory = False)

In [781]:
dataframe.columns.values


array(['csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Round', 'Best of', 'Winner', 'Loser', 'PlayedA', 'PlayedB',
       'WinRatioA', 'WinRatioB', 'WRank', 'WRankAdded', 'LRank',
       'LRankAdded', 'WPts', 'WPtsAdded', 'LPts', 'LPtsAdded', 'W1', 'L1',
       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
       'Last1A', 'Last2A', 'Last3A', 'Last4A', 'Last5A', 'LastSetsA',
       'Last1B', 'Last2B', 'Last3B', 'Last4B', 'Last5B', 'LastSetsB',
       'MaxW', 'MaxWAdded', 'MaxL', 'MaxLAdded', 'AvgW', 'AvgWAdded',
       'AvgL', 'AvgLAdded', 'MinW', 'MinL', 'PlayedCourtA',
       'PlayedCourtB', 'WonRatioCourtA', 'WonRatioCourtB', 'Indoor',
       'Outdoor', 'Clay', 'Grass', 'Hard', 'Awarded', 'Completed',
       'Disqualified', 'Retired', 'Sched', 'Walkover', 'Djokovic N.A',
       'Djokovic N.B', 'Nadal R.A', 'Nadal R.B', 'Federer R.A',
       'Federer R.B', 'Murray A.A', 'Murray A.B', 'Ferrer D.A',
       'Ferrer D.B', 'Berdych T.A', 'Berdych T.

In [782]:
def renameColumns():
    dataframe.rename(columns={'Winner':'PlayerA', 
                          'Loser':'PlayerB',
                          'WRank':'RankA',
                          'WRankAdded':'RankFilledA',
                          'LRank':'RankB',
                          'LRankAdded':'RankFilledB',
                          'W1':'1A',
                          'L1':'1B',
                          'W2':'2A',
                          'L2':'2B',
                          'W3':'3A',
                          'L3':'3B',
                          'W4':'4A',
                          'L4':'4B',
                          'W5':'5A',
                          'L5':'5B',
                          'Wsets':'setsA',
                          'Lsets':'setsB',
                          'WPts':'PtsA',
                          'LPts':'PtsB',
                          'WPtsAdded':'PtsFilledA',
                          'LPtsAdded':'PtsFilledB',
                          'MaxW':'MaxA',
                          'MaxWAdded':'MaxFilledA',
                          'MaxL':'MaxB',
                          'MaxLAdded':'MaxFilledB',
                          'AvgW':'AvgA',
                          'AvgWAdded':'AvgFilledA',
                          'AvgL':'AvgB',
                          'AvgLAdded':'AvgFilledB',
                          'MinW':'MinA',
                          'MinL':'MinB',
                         }, inplace = True)
    dataframe.insert(0, column ='Winner', value = 0)

In [783]:
renameColumns()
dataframe.columns

Index(['Winner', 'csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Round', 'Best of', 'PlayerA',
       ...
       '10_3MeanA', '10_4MeanA', '10_5MeanA', '10_setsMeanA', '10_1MeanB',
       '10_2MeanB', '10_3MeanB', '10_4MeanB', '10_5MeanB', '10_setsMeanB'],
      dtype='object', length=143)

In [784]:
players = dataframe['PlayerA'].value_counts().subtract(
        dataframe['PlayerB'].value_counts(), fill_value = 0).sort_values(ascending = False).index.tolist()[:25]

def getInvertedDataFrame(colToInvert):
    
    colToInvert += players
    
    dataframeInverted = dataframe.copy(deep = True)

    dataframeInverted['Winner'] = 1
    
    for f in colToInvert:
        dataframeInverted[[f+'A', f+'B']] = dataframeInverted[[f+'B', f+'A']]
        
    dataframeInverted['OpponentsWRatioA>OpponentsWRatioB'] = dataframeInverted.apply(lambda x : int(x['OpponentsWRatioA'] > x['OpponentsWRatioB']), axis = 1)
    dataframeInverted['RankA>RankB'] = dataframeInverted.apply(lambda x : int(x['RankA'] > x['RankB']), axis = 1)
    dataframeInverted['WinRatioA>WinRatioB'] = dataframeInverted.apply(lambda x : int(x['WinRatioA'] > x['WinRatioB']), axis = 1)
    dataframeInverted['WonRatioCourtA>WonRatioCourtB'] = dataframeInverted.apply(lambda x : int(x['WonRatioCourtA'] > x['WonRatioCourtB']), axis = 1)
    
    return dataframeInverted

In [785]:
dataframeInverted = getInvertedDataFrame(['Player', 'Rank', 'RankFilled', 'Pts', 'PtsFilled', '1', 
                   '2', '3', '4', '5', 'sets', 'Max', 'Avg', 'Min', 'Other', 'MaxFilled', 'AvgFilled',  'Played',
       'WinRatio', 'WonRatioCourt', 'PlayedCourt', 'OpponentsWRatio', '10_1Mean', '10_2Mean',
                                          '10_3Mean', '10_4Mean', 'Last1', 'Last2', 'Last3', 'Last4', 'Last5', 'LastSets',
               '10_5Mean','10_setsMean'])



In [786]:
dataframeInverted.head()

,Winner,csvID,ATP,Location,Tournament,Date,Series,Round,Best of,PlayerA,...,10_3MeanA,10_4MeanA,10_5MeanA,10_setsMeanA,10_1MeanB,10_2MeanB,10_3MeanB,10_4MeanB,10_5MeanB,10_setsMeanB
0,1,0.0,1.0,13.0,0.0,2010-01-04,0.0,0.0,3.0,Nieminen J.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,1.0,13.0,0.0,2010-01-04,0.0,0.0,3.0,Clement A.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,1.0,13.0,0.0,2010-01-04,0.0,0.0,3.0,Petzschner P.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,1.0,13.0,0.0,2010-01-04,0.0,0.0,3.0,Chardy J.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,1.0,13.0,0.0,2010-01-04,0.0,0.0,3.0,Llodra M.,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [787]:
with pd.option_context('display.max_columns',None):
    display(dataframeInverted[dataframeInverted['PlayerA'] == 'Nadal R.'].tail(2))
    display(dataframe[dataframe['PlayerB'] == 'Nadal R.'].tail(2))
    
#gamesWinRatioTourn pare avere problemi

,Winner,csvID,ATP,Location,Tournament,Date,Series,Round,Best of,PlayerA,PlayerB,PlayedA,PlayedB,WinRatioA,WinRatioB,RankA,RankFilledA,RankB,RankFilledB,PtsA,PtsFilledA,PtsB,PtsFilledB,1A,1B,2A,2B,3A,3B,4A,4B,5A,5B,setsA,setsB,Last1A,Last2A,Last3A,Last4A,Last5A,LastSetsA,Last1B,Last2B,Last3B,Last4B,Last5B,LastSetsB,MaxA,MaxFilledA,MaxB,MaxFilledB,AvgA,AvgFilledA,AvgB,AvgFilledB,MinA,MinB,PlayedCourtA,PlayedCourtB,WonRatioCourtA,WonRatioCourtB,Indoor,Outdoor,Clay,Grass,Hard,Awarded,Completed,Disqualified,Retired,Sched,Walkover,Djokovic N.A,Djokovic N.B,Nadal R.A,Nadal R.B,Federer R.A,Federer R.B,Murray A.A,Murray A.B,Ferrer D.A,Ferrer D.B,Berdych T.A,Berdych T.B,Nishikori K.A,Nishikori K.B,Wawrinka S.A,Wawrinka S.B,Del Potro J.M.A,Del Potro J.M.B,Tsonga J.W.A,Tsonga J.W.B,Cilic M.A,Cilic M.B,Raonic M.A,Raonic M.B,Isner J.A,Isner J.B,Monfils G.A,Monfils G.B,Gasquet R.A,Gasquet R.B,Thiem D.A,Thiem D.B,Anderson K.A,Anderson K.B,Dimitrov G.A,Dimitrov G.B,Zverev A.A,Zverev A.B,Almagro N.A,Almagro N.B,Goffin D.A,Goffin D.B,Kohlschreiber P.A,Kohlschreiber P.B,Simon G.A,Simon G.B,Bautista Agut R.A,Bautista Agut R.B,Verdasco F.A,Verdasco F.B,OtherA,OtherB,RankA>RankB,OpponentsPlayed,OpponentsWRatioA,OpponentsWRatioB,OpponentsWRatioA>OpponentsWRatioB,WonRatioCourtA>WonRatioCourtB,WinRatioA>WinRatioB,10_1MeanA,10_2MeanA,10_3MeanA,10_4MeanA,10_5MeanA,10_setsMeanA,10_1MeanB,10_2MeanB,10_3MeanB,10_4MeanB,10_5MeanB,10_setsMeanB
25293,1,9.0,39.0,40.0,25.0,2019-07-12,4.0,6.0,5.0,Nadal R.,Federer R.,649.0,632.0,0.838213,0.837025,2.0,0.0,3.0,0.0,7945.0,0.0,6620.0,0.0,6.0,7.0,6.0,1.0,3.0,6.0,4.0,6.0,0.0,0.0,1.0,3.0,7.0,6.0,6.0,0.0,0.0,3.0,4.0,6.0,6.0,6.0,0.0,3.0,1.77,0.0,2.30,0.0,1.71,0.0,2.18,0.0,1.71,2.10,52.0,105.0,0.769231,0.876190,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,20.0,0.55,0.45,1,0,1,6.1,5.6,5.9,1.9,0.0,3.0,5.9,5.7,4.4,1.2,0.0,2.5
26255,1,9.0,65.0,62.0,19.0,2019-11-02,2.0,6.0,3.0,Nadal R.,Shapovalov D.,665.0,138.0,0.840602,0.550725,2.0,0.0,28.0,0.0,9225.0,0.0,1460.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,6.0,0.0,0.0,0.0,2.0,6.0,6.0,0.0,0.0,0.0,2.0,1.21,0.0,5.63,0.0,1.18,0.0,4.94,0.0,1.16,4.94,338.0,106.0,0.795858,0.603774,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,2.0,0.50,0.50,0,1,1,5.8,5.2,3.5,1.0,0.6,2.4,4.8,5.0,1.2,0.0,0.0,1.4


,Winner,csvID,ATP,Location,Tournament,Date,Series,Round,Best of,PlayerA,PlayerB,PlayedA,PlayedB,WinRatioA,WinRatioB,RankA,RankFilledA,RankB,RankFilledB,PtsA,PtsFilledA,PtsB,PtsFilledB,1A,1B,2A,2B,3A,3B,4A,4B,5A,5B,setsA,setsB,Last1A,Last2A,Last3A,Last4A,Last5A,LastSetsA,Last1B,Last2B,Last3B,Last4B,Last5B,LastSetsB,MaxA,MaxFilledA,MaxB,MaxFilledB,AvgA,AvgFilledA,AvgB,AvgFilledB,MinA,MinB,PlayedCourtA,PlayedCourtB,WonRatioCourtA,WonRatioCourtB,Indoor,Outdoor,Clay,Grass,Hard,Awarded,Completed,Disqualified,Retired,Sched,Walkover,Djokovic N.A,Djokovic N.B,Nadal R.A,Nadal R.B,Federer R.A,Federer R.B,Murray A.A,Murray A.B,Ferrer D.A,Ferrer D.B,Berdych T.A,Berdych T.B,Nishikori K.A,Nishikori K.B,Wawrinka S.A,Wawrinka S.B,Del Potro J.M.A,Del Potro J.M.B,Tsonga J.W.A,Tsonga J.W.B,Cilic M.A,Cilic M.B,Raonic M.A,Raonic M.B,Isner J.A,Isner J.B,Monfils G.A,Monfils G.B,Gasquet R.A,Gasquet R.B,Thiem D.A,Thiem D.B,Anderson K.A,Anderson K.B,Dimitrov G.A,Dimitrov G.B,Zverev A.A,Zverev A.B,Almagro N.A,Almagro N.B,Goffin D.A,Goffin D.B,Kohlschreiber P.A,Kohlschreiber P.B,Simon G.A,Simon G.B,Bautista Agut R.A,Bautista Agut R.B,Verdasco F.A,Verdasco F.B,OtherA,OtherB,RankA>RankB,OpponentsPlayed,OpponentsWRatioA,OpponentsWRatioB,OpponentsWRatioA>OpponentsWRatioB,WonRatioCourtA>WonRatioCourtB,WinRatioA>WinRatioB,10_1MeanA,10_2MeanA,10_3MeanA,10_4MeanA,10_5MeanA,10_setsMeanA,10_1MeanB,10_2MeanB,10_3MeanB,10_4MeanB,10_5MeanB,10_setsMeanB
25293,0,9.0,39.0,40.0,25.0,2019-07-12,4.0,6.0,5.0,Federer R.,Nadal R.,632.0,649.0,0.837025,0.838213,3.0,0.0,2.0,0.0,6620.0,0.0,7945.0,0.0,7.0,6.0,1.0,6.0,6.0,3.0,6.0,4.0,0.0,0.0,3.0,1.0,4.0,6.0,6.0,6.0,0.0,3.0,7.0,6.0,6.0,0.0,0.0,3.0,2.30,0.0,1.77,0.0,2.18,0.0,1.71,0.0,2.10,1.71,105.0,52.0,0.876190,0.769231,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20.0,0.45,0.55,0.0,1.0,0.0,5.9,5.7,4.4,1.2,0.0,2.5,6.1,5.6,5.9,1.9,0.0,3.0
26255,0,9.0,65.0,62.0,19.0,2019-11-02,2.0,6.0,3.0,Shapovalov D.,Nadal R.,138.0,665.0,0.550725,0.840602,28.0,0.0,2.0,0.0,1460.0,0.0,9225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,2.0,7.0,6.0,0.0,0.0,0.0,2.0,5.63,0.0,1.21,0.0,4.94,0.0,1.18,0.0,4.94,1.16,106.0,338.0,0.603774,0.795858,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.50,0.50,0.0,0.0,0.0,4.8,5.0,1.2,0.0,0.0,1.4,5.8,5.2,3.5,1.0,0.6,2.4


In [788]:
dataframe = dataframe.append(dataframeInverted, sort = False, ignore_index = True)

In [789]:
def sortMatchesByDate ():
    dataframe['Index'] = dataframe.index
    dataframe.sort_values(['csvID','ATP','Index'],inplace = True)
    dataframe.drop('Index', axis=1, inplace = True)
    
sortMatchesByDate()

In [791]:

dataframe.drop([ 'csvID','Date', 'PlayerA', 'PlayerB', '1A', '1B', '2A', '2B', '3A', '3B', '4A', 
                '4B', '5A', '5B', 'setsA', 'setsB', 'Awarded', 'Completed',
       'Disqualified', 'Retired', 'Sched', 'Walkover'], axis = 1, inplace = True)

In [792]:

#PlayedA e B inutili

In [793]:
dataframe = dataframe.reset_index(drop = True)
dataframe.head(50)

,Winner,ATP,Location,Tournament,Series,Round,Best of,PlayedA,PlayedB,WinRatioA,...,10_3MeanA,10_4MeanA,10_5MeanA,10_setsMeanA,10_1MeanB,10_2MeanB,10_3MeanB,10_4MeanB,10_5MeanB,10_setsMeanB
0,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
5,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
6,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
7,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
8,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
9,0,1.0,13.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [794]:
dataframe.to_csv('./datasets/dataframe0.csv', index = False)